In [ ]:
env="dev"
host="redis-apollo-replication-group-c3f7756.ggxdkn.ng.0001.apn2.cache.amazonaws.com"

In [ ]:
redis_info = {}
redis_info["host"] = host
redis_info["port"] = 6379

s3_path = f"s3a://apollo-ci-mapping-{env}/ci_table"
print(s3_path)

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.set("spark.jars", f"s3a://apollo-applog-backup-{env}/spark-redis_2.12-3.0.0-SNAPSHOT-jar-with-dependencies.jar")

sparkConf.set("spark.redis.host", redis_info['host'])
sparkConf.set("spark.redis.port", redis_info['port'])
# sparkConf.set("spark.redis.auth", redis_info['password'])
sparkConf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

In [ ]:
df = spark.read.parquet(s3_path)

In [ ]:
if df.count() > 0:
    df.write.format("org.apache.spark.sql.redis").option("table", "ci").option("key.column", "svc_mgmt_num").save(mode="append")

In [ ]:
spark.stop()